# Azure AI Agent Service - Complete Agent Architecture

**What This Notebook Shows**

- Basic agent creation with Azure AI Foundry Agent Service
- Tool integration - How agents call Python functions
- Event streaming - Capture real-time tool calls and responses
- Production patterns - Async execution and event handling
- Observability - Trace agent decisions and tool usage

**Focus**: End-to-end agent architecture from simple conversation to complex tool orchestration.

## Before you begin

Creating an agent using Azure AI agent service requires an Azure AI Foundry project and a deployed, supported model. See more details in [Create a new agent](https://learn.microsoft.com/azure/ai-services/agents/quickstart?pivots=ai-foundry-portal).

For quality evaluation, you need to deploy a `gpt` model supporting JSON mode. We recommend a model `gpt-4o` or `gpt-4o-mini` for their strong reasoning capabilities. You should expect to spend about 20 minutes running this notebook.     

Important: Make sure to authenticate to Azure using `az login` in your terminal before running this notebook.

Also, ensure you have a blob storage account with configured RBAC access for the AI Foundry Projects identity: https://github.com/MicrosoftDocs/azure-ai-docs/blob/main/articles/ai-foundry/how-to/evaluations-storage-account.md

## Prerequisites

Before running the sample:
```bash
pip install azure-ai-projects azure-identity azure-ai-evaluation
```
Set these environment variables with your own values:
1) **PROJECT_CONNECTION_STRING** - The project connection string, as found in the overview page of your Azure AI Foundry project.
2) **MODEL_DEPLOYMENT_NAME** - The deployment name of the model for AI-assisted evaluators, as found under the "Name" column in the "Models + endpoints" tab in your Azure AI Foundry project.
3) **AZURE_OPENAI_ENDPOINT** - Azure Open AI Endpoint to be used for evaluation.
4) **AZURE_OPENAI_API_KEY** - Azure Open AI Key to be used for evaluation.
5) **AZURE_OPENAI_API_VERSION** - Azure Open AI Api version to be used for evaluation.
6) **AZURE_SUBSCRIPTION_ID** - Azure Subscription Id of Azure AI Project
7) **PROJECT_NAME** - Azure AI Project Name
8) **RESOURCE_GROUP_NAME** - Azure AI Project Resource Group Name
9) **AGENT_MODEL_DEPLOYMENT_NAME** - The deployment name of the model for your Azure AI agent, as found under the "Name" column in the "Models + endpoints" tab in your Azure AI Foundry project.

## Agent Development Flow

**This notebook demonstrates the complete agent architecture in logical steps:**

1. **Define Tools** → Create Python functions that agents can call
2. **Create Agent** → One-time setup with tools attached  
3. **Simple Execution** → Basic tool-enabled conversation
4. **Tool Monitoring** → See exactly which tools are called and when
5. **Streaming** → Real-time responses with tool visibility
6. **Async Events** → Production-ready event handling

**Key Pattern**: Define tools → Create agent once → Reuse agent ID for all interactions

**Engineer Focus**: Concise, production-ready patterns without over-engineering.

In [1]:
# 📂 Setup Working Directory for ARTAgent Framework Access
import logging
import os

# Configure logging to track directory changes
logging.basicConfig(level=logging.INFO)

# Navigate to the project root directory
# This ensures we can import ARTAgent framework modules properly
try:
    # Move up two directories from samples/hello_world/ to project root
    os.chdir("../../")
    
    # Allow override via environment variable for different setups
    target_directory = os.getenv(
        "TARGET_DIRECTORY", os.getcwd()
    )  # Use environment variable if available
    
    # Verify the target directory exists before changing
    if os.path.exists(target_directory):
        os.chdir(target_directory)
        print(f"✅ Changed directory to: {os.getcwd()}")
        logging.info(f"Successfully changed directory to: {os.getcwd()}")
    else:
        print(f"❌ Directory does not exist: {target_directory}")
        logging.error(f"Directory does not exist: {target_directory}")
        
except Exception as e:
    print(f"❌ Error changing directory: {e}")
    logging.exception(f"An error occurred while changing directory: {e}")

# Verify we're in the correct location
print(f"📁 Current working directory: {os.getcwd()}")
print(f"📋 Contents: {', '.join(os.listdir('.')[:10])}...")

INFO:root:Successfully changed directory to: c:\Users\pablosal\Desktop\gbb-ai-audio-agent


✅ Changed directory to: c:\Users\pablosal\Desktop\gbb-ai-audio-agent
📁 Current working directory: c:\Users\pablosal\Desktop\gbb-ai-audio-agent
📋 Contents: .azure, .devcontainer, .env, .env.aoai_pool, .env.sample, .files, .git, .github, .gitignore, .pre-commit-config.yaml...


## Initializing Project Client

In [ ]:
import os
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.agents.models import FunctionTool, ToolSet

# Import your custom functions to be used as Tools for the Agent
from samples.hello_world.foundryagents.tool_store.user_functions import user_functions

project_client = AIProjectClient(
    credential=DefaultAzureCredential(),
    endpoint=os.environ["AZURE_AI_FOUNDRY_ENDPOINT"],
)

# Add Tools to be used by Agent
functions = FunctionTool(user_functions)

toolset = ToolSet()
toolset.add(functions)

# To enable tool calls executed automatically
project_client.agents.enable_auto_function_calls(tools=toolset)

INFO:azure.identity._credentials.environment:No environment configuration found.
INFO:azure.identity._credentials.managed_identity:ManagedIdentityCredential will use IMDS
INFO:azure.identity._credentials.managed_identity:ManagedIdentityCredential will use IMDS


## Create an AI Agent (Azure AI Agent Service)

In [3]:
AGENT_NAME = "Seattle Tourist Assistant"
# Concise instructions for the Seattle Tourist Assistant
INSTRUCTIONS_PROMPT = """
You are the Seattle Tourist Assistant — a friendly, concise travel helper for visitors to Seattle, WA.
- Use available tools for facts (fetch_weather, opening_hours, send_email, convert_temperature, calculate_sum).
- For factual requests, call the matching tool and prefer tool output over guessing.
- When sending email: draft subject & body, ask for recipient confirmation, then call send_email.
- Start replies with a one-line summary, use bullets for options, and finish with a next-step suggestion.
- Ask one clarifying question if info is missing.
""".strip()
MODEL_NAME= "gpt-4o"

In [4]:
agent = project_client.agents.create_agent(
    model=MODEL_NAME,
    name=AGENT_NAME,
    instructions=INSTRUCTIONS_PROMPT,
    toolset=toolset,
)


print(f"Created agent, ID: {agent.id}")

INFO:azure.core.pipeline.policies.http_logging_policy:Request URL: 'http://169.254.169.254/metadata/identity/oauth2/token?api-version=REDACTED&resource=REDACTED'
Request method: 'GET'
Request headers:
    'User-Agent': 'azsdk-python-identity/1.19.0 Python/3.11.11 (Windows-10-10.0.26100-SP0)'
No body was attached to the request
INFO:azure.identity._credentials.chained:DefaultAzureCredential acquired a token from AzureCliCredential
INFO:azure.core.pipeline.policies.http_logging_policy:Request URL: 'https://poc-ai-agents-voice-resource.services.ai.azure.com/api/projects/poc-ai-agents-voice/assistants?api-version=REDACTED'
Request method: 'POST'
Request headers:
    'Content-Type': 'application/json'
    'Content-Length': '4262'
    'Accept': 'application/json'
    'x-ms-client-request-id': 'd35cedf3-8854-11f0-87de-f43bd8cfe843'
    'User-Agent': 'AIProjectClient azsdk-python-ai-agents/1.1.0 Python/3.11.11 (Windows-10-10.0.26100-SP0)'
    'Authorization': 'REDACTED'
A body is sent with the

Created agent, ID: asst_g7HbsvkaqXwYtA5VU9sdKZJ6


## Adding Tools to Azure AI Foundry Agents

**Real Agent Architecture**: Agents need external capabilities to be useful. Tools are Python functions that agents can call to:

- **Access data** (APIs, databases, files)
- **Perform calculations** (math, analysis, processing) 
- **Take actions** (send emails, create records, trigger workflows)

**Key Pattern**: Agent receives request → Reasons about what tools to use → Calls tools → Uses results to generate response

Let's create practical tools that demonstrate this pattern:

In [5]:
from typing import Callable, Dict, Any, Set, List

# Example tools that demonstrate real agent capabilities
def get_stock_price(symbol: str) -> float:
    """Get current stock price for a given symbol."""
    # In production: call actual API like Alpha Vantage, Yahoo Finance
    prices = {"TSLA": 245.67, "MSFT": 415.23, "AAPL": 189.84}
    return prices.get(symbol.upper(), 123.45)

def analyze_sentiment(text: str) -> Dict[str, Any]:
    """Analyze sentiment of text using ML model."""
    # In production: call Azure Text Analytics or custom model
    positive_words = ["good", "great", "excellent", "amazing", "positive"]
    negative_words = ["bad", "terrible", "awful", "negative", "horrible"]
    
    text_lower = text.lower()
    pos_count = sum(1 for word in positive_words if word in text_lower)
    neg_count = sum(1 for word in negative_words if word in text_lower)
    
    if pos_count > neg_count:
        return {"sentiment": "positive", "score": 0.85, "confidence": "high"}
    elif neg_count > pos_count:
        return {"sentiment": "negative", "score": 0.15, "confidence": "high"}
    else:
        return {"sentiment": "neutral", "score": 0.50, "confidence": "medium"}

def summarize_text(text: str, max_length: int = 100) -> str:
    """Summarize text to specified length."""
    # In production: use Azure OpenAI summarization or custom model
    if len(text) <= max_length:
        return text
    return f"{text[:max_length-3]}..."

def calculate_metrics(data: List[float]) -> Dict[str, float]:
    """Calculate basic statistical metrics."""
    if not data:
        return {"error": "No data provided"}
    
    return {
        "count": len(data),
        "sum": sum(data),
        "average": sum(data) / len(data),
        "min": min(data),
        "max": max(data)
    }

# Tool registry - Azure AI Foundry will discover these functions
AGENT_TOOLS: Set[Callable[..., Any]] = {
    get_stock_price,
    analyze_sentiment, 
    summarize_text,
    calculate_metrics
}

print("✅ Agent tools defined:")
for tool in AGENT_TOOLS:
    print(f"   • {tool.__name__}: {tool.__doc__}")

✅ Agent tools defined:
   • summarize_text: Summarize text to specified length.
   • calculate_metrics: Calculate basic statistical metrics.
   • analyze_sentiment: Analyze sentiment of text using ML model.
   • get_stock_price: Get current stock price for a given symbol.


In [8]:
import sys
import json
import time
import functools
from utils.ml_logging import get_logger
from azure.core.exceptions import HttpResponseError
from azure.identity import DefaultAzureCredential
from azure.ai.agents import AgentsClient
from azure.ai.agents.models import (
    FunctionTool,
    ToolSet,
    ToolOutput,
    RequiredFunctionToolCall,
    SubmitToolOutputsAction,
    RunStatus,
    MessageTextContent,
    ListSortOrder,
)

logger = get_logger()

def get_agents_client() -> AgentsClient:
    """Return an authenticated AgentsClient based on env vars."""
    endpoint = os.getenv("AZURE_AI_FOUNDRY_URL") or os.getenv("AZURE_AI_FOUNDRY_ENDPOINT")
    if not endpoint:
        logger.error("AZURE_AI_FOUNDRY_URL or AZURE_AI_FOUNDRY_ENDPOINT must be set")
        sys.exit(1)
    cred = DefaultAzureCredential()
    return AgentsClient(endpoint=endpoint, credential=cred)

def get_model_deployment() -> str:
    """Return the model deployment name from env vars."""
    dep = (os.getenv("AZURE_AOAI_CHAT_MODEL_NAME_DEPLOYMENT_ID") or 
           os.getenv("AGENT_MODEL_DEPLOYMENT_NAME") or 
           "gpt-4o")
    return dep

def create_agent(client: AgentsClient, deployment: str, toolset: ToolSet | None = None) -> str:
    """Create an agent (optionally with tools) and return its ID."""
    try:
        agent = client.create_agent(
            model=deployment,
            name="demo-agent",
            instructions="You are a concise assistant that uses available tools to help users.",
            toolset=toolset,
        )
        logger.info("Agent created: %s", agent.id)
        return agent.id
    except HttpResponseError as e:
        logger.error("Agent creation failed: %s", e)
        sys.exit(1)

def create_thread(client: AgentsClient) -> str:
    """Start a new conversation thread."""
    t = client.threads.create()
    logger.info("Thread created: %s", t.id)
    return t.id

def post_message(client: AgentsClient, thread_id: str, role: str, text: str) -> None:
    """Post a message to a thread."""
    m = client.messages.create(thread_id=thread_id, role=role, content=text)
    logger.debug("Message posted: %s", m.id)

def json_safe_wrapper(func):
    """Wrap tool functions to return JSON strings - required for Azure AI Foundry."""
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        # Always return JSON string
        if isinstance(result, dict):
            return json.dumps(result)
        else:
            return json.dumps({"value": result})
    return wrapper

print("✅ Helper functions defined for agent management")


✅ Helper functions defined for agent management



In [ ]:
## **Step 1: Create Agent with Tools** (One-time setup)

# Create JSON-safe versions of all tools (Azure AI Foundry requires JSON string outputs)
safe_agent_tools = {json_safe_wrapper(tool) for tool in AGENT_TOOLS}

# Create agent with JSON-safe tools - we'll reuse this agent for all examples
func_tool = FunctionTool(safe_agent_tools)
toolset = ToolSet()
toolset.add(func_tool)

print("🤖 Creating agent with JSON-safe tools...")
client = get_agents_client()
deploy = get_model_deployment()

# Create the agent ONCE - we'll reuse this ID
agent_id = create_agent(client, deploy, toolset)
print(f"✅ Agent created: {agent_id}")

# Create conversation thread 
thread_id = create_thread(client)
print(f"✅ Thread created: {thread_id}")

print("\n🎯 Agent setup complete! Agent ID will be reused for all examples below.")

🤖 Creating agent with JSON-safe tools...



INFO:azure.identity._credentials.environment:No environment configuration found.
INFO:azure.identity._credentials.managed_identity:ManagedIdentityCredential will use IMDS
INFO:azure.core.pipeline.policies.http_logging_policy:Request URL: 'http://169.254.169.254/metadata/identity/oauth2/token?api-version=REDACTED&resource=REDACTED'
Request method: 'GET'
Request headers:
    'User-Agent': 'azsdk-python-identity/1.19.0 Python/3.11.11 (Windows-10-10.0.26100-SP0)'
No body was attached to the request
INFO:azure.identity._credentials.managed_identity:ManagedIdentityCredential will use IMDS
INFO:azure.core.pipeline.policies.http_logging_policy:Request URL: 'http://169.254.169.254/metadata/identity/oauth2/token?api-version=REDACTED&resource=REDACTED'
Request method: 'GET'
Request headers:
    'User-Agent': 'azsdk-python-identity/1.19.0 Python/3.11.11 (Windows-10-10.0.26100-SP0)'
No body was attached to the request
INFO:azure.identity._credentials.chained:DefaultAzureCredential acquired a token 

✅ Agent created: asst_luun9AZWwRQCiXDDrWM8Fdnc



INFO:azure.core.pipeline.policies.http_logging_policy:Request URL: 'https://poc-ai-agents-voice-resource.services.ai.azure.com/api/projects/poc-ai-agents-voice/threads?api-version=REDACTED'
Request method: 'POST'
Request headers:
    'Content-Type': 'application/json'
    'Content-Length': '2'
    'Accept': 'application/json'
    'x-ms-client-request-id': 'ea691eb0-8854-11f0-9516-f43bd8cfe843'
    'User-Agent': 'azsdk-python-ai-agents/1.1.0 Python/3.11.11 (Windows-10-10.0.26100-SP0)'
    'Authorization': 'REDACTED'
A body is sent with the request
INFO:azure.core.pipeline.policies.http_logging_policy:Response status: 200
Response headers:
    'Content-Length': '137'
    'Content-Type': 'application/json'
    'Request-Context': 'REDACTED'
    'x-ms-response-type': 'REDACTED'
    'x-ms-middleware-request-id': 'REDACTED'
    'openai-version': 'REDACTED'
    'openai-organization': 'REDACTED'
    'X-Request-ID': 'REDACTED'
    'openai-processing-ms': 'REDACTED'
    'Strict-Transport-Security

✅ Thread created: thread_oVN6OQULpl3S7GC5DqLPu7co

🎯 Agent setup complete! Agent ID will be reused for all examples below.
🔧 All tools wrapped with JSON serialization for Azure AI Foundry compatibility.


🎯 Agent setup complete! Agent ID will be reused for all examples below.
🔧 All tools wrapped with JSON serialization for Azure AI Foundry compatibility.


In [10]:
## **Step 2: Simple Agent Execution** (Using existing agent)

def run_simple_agent(client, agent_id, thread_id, user_message):
    """Simple agent run - shows basic tool usage without complex event handling."""
    
    print(f"📝 User: {user_message}")
    
    # Send message to agent
    post_message(client, thread_id, "user", user_message)
    
    # Run agent (this handles tool calls automatically)
    run = client.runs.create_and_process(thread_id=thread_id, agent_id=agent_id)
    print(f"🔄 Agent finished with status: {run.status}")
    
    # Get the conversation
    print("\n--- Agent Response ---")
    messages = client.messages.list(thread_id, order=ListSortOrder.ASCENDING)
    
    for msg in messages:
        if msg.text_messages:
            last = msg.text_messages[-1]
            if isinstance(last, MessageTextContent):
                role = "YOU" if msg.role == "user" else "AGENT"
                print(f"{role}: {last.text.value}")
    
    return run

# Test the agent with a simple request (reusing existing agent_id and thread_id)
print("=" * 60)
print("STEP 2: SIMPLE EXECUTION")
print("=" * 60)

simple_result = run_simple_agent(
    client, 
    agent_id, 
    thread_id,
    "Get the current TSLA stock price and tell me the sentiment of 'Tesla is doing great this quarter'"
)

STEP 2: SIMPLE EXECUTION
📝 User: Get the current TSLA stock price and tell me the sentiment of 'Tesla is doing great this quarter'

STEP 2: SIMPLE EXECUTION
📝 User: Get the current TSLA stock price and tell me the sentiment of 'Tesla is doing great this quarter'


INFO:azure.core.pipeline.policies.http_logging_policy:Request URL: 'https://poc-ai-agents-voice-resource.services.ai.azure.com/api/projects/poc-ai-agents-voice/threads/thread_oVN6OQULpl3S7GC5DqLPu7co/messages?api-version=REDACTED'
Request method: 'POST'
Request headers:
    'Content-Type': 'application/json'
    'Content-Length': '128'
    'Accept': 'application/json'
    'x-ms-client-request-id': 'f3ac6180-8854-11f0-819d-f43bd8cfe843'
    'User-Agent': 'azsdk-python-ai-agents/1.1.0 Python/3.11.11 (Windows-10-10.0.26100-SP0)'
    'Authorization': 'REDACTED'
A body is sent with the request
INFO:azure.core.pipeline.policies.http_logging_policy:Response status: 200
Response headers:
    'Transfer-Encoding': 'chunked'
    'Content-Type': 'application/json'
    'Content-Encoding': 'REDACTED'
    'Vary': 'REDACTED'
    'Request-Context': 'REDACTED'
    'x-ms-response-type': 'REDACTED'
    'x-ms-middleware-request-id': 'REDACTED'
    'openai-version': 'REDACTED'
    'openai-organization': 'RE

🔄 Agent finished with status: RunStatus.COMPLETED

--- Agent Response ---


--- Agent Response ---


INFO:azure.core.pipeline.policies.http_logging_policy:Request URL: 'https://poc-ai-agents-voice-resource.services.ai.azure.com/api/projects/poc-ai-agents-voice/threads/thread_oVN6OQULpl3S7GC5DqLPu7co/messages?api-version=REDACTED&order=REDACTED'
Request method: 'GET'
Request headers:
    'Accept': 'application/json'
    'x-ms-client-request-id': 'f5fc29e1-8854-11f0-8699-f43bd8cfe843'
    'User-Agent': 'azsdk-python-ai-agents/1.1.0 Python/3.11.11 (Windows-10-10.0.26100-SP0)'
    'Authorization': 'REDACTED'
No body was attached to the request
INFO:azure.core.pipeline.policies.http_logging_policy:Response status: 200
Response headers:
    'Transfer-Encoding': 'chunked'
    'Content-Type': 'application/json'
    'Content-Encoding': 'REDACTED'
    'Vary': 'REDACTED'
    'Request-Context': 'REDACTED'
    'x-ms-response-type': 'REDACTED'
    'x-ms-middleware-request-id': 'REDACTED'
    'openai-version': 'REDACTED'
    'openai-organization': 'REDACTED'
    'X-Request-ID': 'REDACTED'
    'opena

YOU: Get the current TSLA stock price and tell me the sentiment of 'Tesla is doing great this quarter'
AGENT: It seems I'm unable to retrieve the TSLA stock price or analyze the sentiment at the moment. Let me know if you'd like assistance with something else.

AGENT: It seems I'm unable to retrieve the TSLA stock price or analyze the sentiment at the moment. Let me know if you'd like assistance with something else.


INFO:azure.core.pipeline.policies.http_logging_policy:Request URL: 'https://poc-ai-agents-voice-resource.services.ai.azure.com/api/projects/poc-ai-agents-voice/threads/thread_oVN6OQULpl3S7GC5DqLPu7co/messages?api-version=REDACTED&order=REDACTED&after=REDACTED'
Request method: 'GET'
Request headers:
    'Accept': 'application/json'
    'x-ms-client-request-id': 'f6305f67-8854-11f0-a53b-f43bd8cfe843'
    'User-Agent': 'azsdk-python-ai-agents/1.1.0 Python/3.11.11 (Windows-10-10.0.26100-SP0)'
    'Authorization': 'REDACTED'
No body was attached to the request
INFO:azure.core.pipeline.policies.http_logging_policy:Response status: 200
Response headers:
    'Content-Length': '96'
    'Content-Type': 'application/json'
    'Request-Context': 'REDACTED'
    'x-ms-response-type': 'REDACTED'
    'x-ms-middleware-request-id': 'REDACTED'
    'openai-version': 'REDACTED'
    'openai-organization': 'REDACTED'
    'X-Request-ID': 'REDACTED'
    'openai-processing-ms': 'REDACTED'
    'Strict-Transport-

In [12]:
## **Step 3: Detailed Tool Monitoring** (Using existing agent)

def run_agent_with_tool_monitoring(client, agent_id, thread_id, user_message):
    """Run agent with detailed tool call monitoring - see the tool execution process."""
    
    print(f"📝 User: {user_message}")
    
    # Create JSON-safe tools for this run
    safe_tools = {json_safe_wrapper(tool) for tool in AGENT_TOOLS}
    monitoring_func_tool = FunctionTool(safe_tools)
    
    # Send message
    post_message(client, thread_id, "user", user_message)
    
    # Create run (don't auto-process - we want to see tool calls)
    run = client.runs.create(thread_id=thread_id, agent_id=agent_id)
    print(f"🚀 Run started: {run.id}")
    
    # Monitor the run status and tool calls
    terminal_states = {RunStatus.COMPLETED, RunStatus.FAILED, RunStatus.CANCELLED, RunStatus.EXPIRED}
    
    while run.status not in terminal_states:
        print(f"🔄 Status: {run.status}")
        
        # Check if agent wants to use tools
        if run.status == RunStatus.REQUIRES_ACTION:
            action = run.required_action
            if isinstance(action, SubmitToolOutputsAction):
                tool_calls = action.submit_tool_outputs.tool_calls
                print(f"\n🛠️  Agent wants to call {len(tool_calls)} tool(s):")
                
                tool_outputs = []
                for i, call in enumerate(tool_calls, 1):
                    if isinstance(call, RequiredFunctionToolCall):
                        fn_name = call.function.name
                        args = call.function.arguments
                        print(f"   {i}. {fn_name}({args})")
                        
                        # Execute the tool (with JSON safety)
                        result = monitoring_func_tool.execute(call)
                        print(f"      → Result: {str(result)[:100]}{'...' if len(str(result)) > 100 else ''}")
                        
                        tool_outputs.append(ToolOutput(
                            tool_call_id=call.id,
                            output=result  # Already JSON string from wrapper
                        ))
                
                # Send results back to agent
                client.runs.submit_tool_outputs(
                    thread_id=thread_id,
                    run_id=run.id,
                    tool_outputs=tool_outputs
                )
                print(f"✅ Submitted {len(tool_outputs)} tool results")
        
        time.sleep(1)
        run = client.runs.get(thread_id, run.id)
    
    print(f"\n🎯 Final status: {run.status}")
    return run

# Test with tool monitoring (reusing existing agent_id and thread_id)
print("=" * 60)
print("STEP 3: DETAILED TOOL MONITORING")
print("=" * 60)

monitored_result = run_agent_with_tool_monitoring(
    client,
    agent_id,
    thread_id,
    "Get the current TSLA stock price and tell me the sentiment of 'Tesla is doing great this quarter'"
)

STEP 3: DETAILED TOOL MONITORING
📝 User: Get the current TSLA stock price and tell me the sentiment of 'Tesla is doing great this quarter'

STEP 3: DETAILED TOOL MONITORING
📝 User: Get the current TSLA stock price and tell me the sentiment of 'Tesla is doing great this quarter'


INFO:azure.core.pipeline.policies.http_logging_policy:Request URL: 'https://poc-ai-agents-voice-resource.services.ai.azure.com/api/projects/poc-ai-agents-voice/threads/thread_oVN6OQULpl3S7GC5DqLPu7co/messages?api-version=REDACTED'
Request method: 'POST'
Request headers:
    'Content-Type': 'application/json'
    'Content-Length': '128'
    'Accept': 'application/json'
    'x-ms-client-request-id': '31524e0a-8855-11f0-a4d2-f43bd8cfe843'
    'User-Agent': 'azsdk-python-ai-agents/1.1.0 Python/3.11.11 (Windows-10-10.0.26100-SP0)'
    'Authorization': 'REDACTED'
A body is sent with the request
INFO:azure.core.pipeline.policies.http_logging_policy:Response status: 200
Response headers:
    'Transfer-Encoding': 'chunked'
    'Content-Type': 'application/json'
    'Content-Encoding': 'REDACTED'
    'Vary': 'REDACTED'
    'Request-Context': 'REDACTED'
    'x-ms-response-type': 'REDACTED'
    'x-ms-middleware-request-id': 'REDACTED'
    'openai-version': 'REDACTED'
    'openai-organization': 'RE

🚀 Run started: run_KM7VndWFA7tz9b0OuA1IPfUj
🔄 Status: RunStatus.QUEUED

🔄 Status: RunStatus.QUEUED


INFO:azure.core.pipeline.policies.http_logging_policy:Request URL: 'https://poc-ai-agents-voice-resource.services.ai.azure.com/api/projects/poc-ai-agents-voice/threads/thread_oVN6OQULpl3S7GC5DqLPu7co/runs/run_KM7VndWFA7tz9b0OuA1IPfUj?api-version=REDACTED'
Request method: 'GET'
Request headers:
    'Accept': 'application/json'
    'x-ms-client-request-id': '32550492-8855-11f0-993b-f43bd8cfe843'
    'User-Agent': 'azsdk-python-ai-agents/1.1.0 Python/3.11.11 (Windows-10-10.0.26100-SP0)'
    'Authorization': 'REDACTED'
No body was attached to the request
INFO:azure.core.pipeline.policies.http_logging_policy:Response status: 200
Response headers:
    'Transfer-Encoding': 'chunked'
    'Content-Type': 'application/json'
    'Content-Encoding': 'REDACTED'
    'Vary': 'REDACTED'
    'Request-Context': 'REDACTED'
    'x-ms-response-type': 'REDACTED'
    'x-ms-middleware-request-id': 'REDACTED'
    'openai-version': 'REDACTED'
    'openai-organization': 'REDACTED'
    'X-Request-ID': 'REDACTED'


🔄 Status: RunStatus.REQUIRES_ACTION

🛠️  Agent wants to call 2 tool(s):
   1. get_stock_price({"symbol": "TSLA"})
      → Result: {"value": 245.67}
   2. analyze_sentiment({"text": "Tesla is doing great this quarter"})
      → Result: {"sentiment": "positive", "score": 0.85, "confidence": "high"}


🛠️  Agent wants to call 2 tool(s):
   1. get_stock_price({"symbol": "TSLA"})
      → Result: {"value": 245.67}
   2. analyze_sentiment({"text": "Tesla is doing great this quarter"})
      → Result: {"sentiment": "positive", "score": 0.85, "confidence": "high"}


INFO:azure.core.pipeline.policies.http_logging_policy:Request URL: 'https://poc-ai-agents-voice-resource.services.ai.azure.com/api/projects/poc-ai-agents-voice/threads/thread_oVN6OQULpl3S7GC5DqLPu7co/runs/run_KM7VndWFA7tz9b0OuA1IPfUj/submit_tool_outputs?api-version=REDACTED'
Request method: 'POST'
Request headers:
    'Content-Type': 'application/json'
    'Content-Length': '256'
    'Accept': 'application/json'
    'x-ms-client-request-id': '327df7b9-8855-11f0-8e57-f43bd8cfe843'
    'User-Agent': 'azsdk-python-ai-agents/1.1.0 Python/3.11.11 (Windows-10-10.0.26100-SP0)'
    'Authorization': 'REDACTED'
A body is sent with the request
INFO:azure.core.pipeline.policies.http_logging_policy:Response status: 200
Response headers:
    'Transfer-Encoding': 'chunked'
    'Content-Type': 'application/json'
    'Content-Encoding': 'REDACTED'
    'Vary': 'REDACTED'
    'Request-Context': 'REDACTED'
    'x-ms-response-type': 'REDACTED'
    'x-ms-middleware-request-id': 'REDACTED'
    'openai-versio

✅ Submitted 2 tool results



INFO:azure.core.pipeline.policies.http_logging_policy:Request URL: 'https://poc-ai-agents-voice-resource.services.ai.azure.com/api/projects/poc-ai-agents-voice/threads/thread_oVN6OQULpl3S7GC5DqLPu7co/runs/run_KM7VndWFA7tz9b0OuA1IPfUj?api-version=REDACTED'
Request method: 'GET'
Request headers:
    'Accept': 'application/json'
    'x-ms-client-request-id': '3373328c-8855-11f0-8001-f43bd8cfe843'
    'User-Agent': 'azsdk-python-ai-agents/1.1.0 Python/3.11.11 (Windows-10-10.0.26100-SP0)'
    'Authorization': 'REDACTED'
No body was attached to the request
INFO:azure.core.pipeline.policies.http_logging_policy:Response status: 200
Response headers:
    'Transfer-Encoding': 'chunked'
    'Content-Type': 'application/json'
    'Content-Encoding': 'REDACTED'
    'Vary': 'REDACTED'
    'Request-Context': 'REDACTED'
    'x-ms-response-type': 'REDACTED'
    'x-ms-middleware-request-id': 'REDACTED'
    'openai-version': 'REDACTED'
    'openai-organization': 'REDACTED'
    'X-Request-ID': 'REDACTED'



🎯 Final status: RunStatus.COMPLETED



## Real-Time Streaming & Event-Driven Patterns

Now let's see the **production patterns** - streaming responses and event-driven architecture.

**Why This Matters**:
- **Real-time UIs** - See responses as they generate (like ChatGPT)
- **Tool monitoring** - Watch tool calls happen in real-time
- **Error handling** - Catch problems immediately
- **Scalability** - Handle multiple conversations concurrently

### Streaming with Tool Visibility

In [14]:
## **Step 4: Streaming with Tool Visibility** (Using existing agent)

from azure.ai.agents.models import (
    AgentStreamEvent,
    MessageDeltaChunk,
    RunStepDeltaChunk,
)

def run_streaming_agent(client, agent_id, user_message):
    """Stream agent responses with real-time tool call visibility."""
    
    print(f"📝 User: {user_message}")
    print("🌊 Streaming response...")
    print("-" * 50)
    
    # Create JSON-safe tools for streaming
    safe_tools = {json_safe_wrapper(tool) for tool in AGENT_TOOLS}
    streaming_func_tool = FunctionTool(safe_tools)
    streaming_toolset = ToolSet()
    streaming_toolset.add(streaming_func_tool)
    
    # Create new thread for this conversation
    thread = client.threads.create()
    client.messages.create(thread_id=thread.id, role="user", content=user_message)
    
    # Enable auto tool calls for simpler streaming
    client.enable_auto_function_calls(streaming_toolset)
    
    # Stream the conversation
    with client.runs.stream(thread_id=thread.id, agent_id=agent_id) as stream:
        for event_type, data, _ in stream:
            
            if isinstance(data, MessageDeltaChunk):
                # Stream text tokens
                print(data.text, end="", flush=True)
            
            elif isinstance(data, RunStepDeltaChunk):
                # Show tool calls
                delta = data.delta
                fn_name = (delta.get("function_call", {}).get("name") or 
                          delta.get("tool_call", {}).get("function", {}).get("name"))
                if fn_name:
                    print(f"\n🛠️  Calling: {fn_name}...", flush=True)
            
            elif event_type == AgentStreamEvent.ERROR:
                print(f"\n🚨 Error: {data}")
            
            elif event_type == AgentStreamEvent.DONE:
                print(f"\n✅ Complete!")
    
    print("\n" + "-" * 50)

# Test streaming (reusing existing agent_id)
print("=" * 60)
print("STEP 4: STREAMING WITH TOOL VISIBILITY")
print("=" * 60)

run_streaming_agent(
    client,
    agent_id, 
    "Get TESLA stock price and analyze sentiment of 'Microsoft Azure is excellent for AI development'"
)

STEP 4: STREAMING WITH TOOL VISIBILITY
STEP 4: STREAMING WITH TOOL VISIBILITY

📝 User: Get TESLA stock price and analyze sentiment of 'Microsoft Azure is excellent for AI development'
🌊 Streaming response...
--------------------------------------------------
📝 User: Get TESLA stock price and analyze sentiment of 'Microsoft Azure is excellent for AI development'
🌊 Streaming response...
--------------------------------------------------


INFO:azure.core.pipeline.policies.http_logging_policy:Request URL: 'https://poc-ai-agents-voice-resource.services.ai.azure.com/api/projects/poc-ai-agents-voice/threads?api-version=REDACTED'
Request method: 'POST'
Request headers:
    'Content-Type': 'application/json'
    'Content-Length': '2'
    'Accept': 'application/json'
    'x-ms-client-request-id': '4f22f46a-8855-11f0-92ff-f43bd8cfe843'
    'User-Agent': 'azsdk-python-ai-agents/1.1.0 Python/3.11.11 (Windows-10-10.0.26100-SP0)'
    'Authorization': 'REDACTED'
A body is sent with the request
INFO:azure.core.pipeline.policies.http_logging_policy:Response status: 200
Response headers:
    'Content-Length': '137'
    'Content-Type': 'application/json'
    'Request-Context': 'REDACTED'
    'x-ms-response-type': 'REDACTED'
    'x-ms-middleware-request-id': 'REDACTED'
    'openai-version': 'REDACTED'
    'openai-organization': 'REDACTED'
    'X-Request-ID': 'REDACTED'
    'openai-processing-ms': 'REDACTED'
    'Strict-Transport-Security


✅ Complete!

- The current- The current stock price of Tesla stock price of Tesla (TSLA (TSLA) is $245) is $245.67.67..  
  
- The- The sentiment sentiment of of "Microsoft "Microsoft Azure is Azure is excellent for AI excellent for AI development development" is **" is **positive**positive** with a high with a high confidence confidence and a and a score score of of 0.85. 0.85.
✅ Complete!

--------------------------------------------------

✅ Complete!

--------------------------------------------------


In [ ]:
# Clean up resources (delete the agent we created)
client.delete_agent(agent_id)
print(f"🧹 Agent {agent_id} deleted")

INFO:azure.core.pipeline.policies.http_logging_policy:Request URL: 'https://poc-ai-agents-voice-resource.services.ai.azure.com/api/projects/poc-ai-agents-voice/assistants/asst_luun9AZWwRQCiXDDrWM8Fdnc?api-version=REDACTED'
Request method: 'DELETE'
Request headers:
    'Accept': 'application/json'
    'x-ms-client-request-id': '57dfac5c-8855-11f0-b8d6-f43bd8cfe843'
    'User-Agent': 'azsdk-python-ai-agents/1.1.0 Python/3.11.11 (Windows-10-10.0.26100-SP0)'
    'Authorization': 'REDACTED'
No body was attached to the request
INFO:azure.core.pipeline.policies.http_logging_policy:Response status: 200
Response headers:
    'Content-Length': '95'
    'Content-Type': 'application/json'
    'Request-Context': 'REDACTED'
    'x-ms-response-type': 'REDACTED'
    'x-ms-middleware-request-id': 'REDACTED'
    'openai-version': 'REDACTED'
    'openai-organization': 'REDACTED'
    'X-Request-ID': 'REDACTED'
    'openai-processing-ms': 'REDACTED'
    'Strict-Transport-Security': 'REDACTED'
    'apim-req

🧹 Agent asst_luun9AZWwRQCiXDDrWM8Fdnc deleted
✅ Cleanup complete

✅ Cleanup complete


## Async + Event Handlers

**Production Requirements**:
- **Async execution** - Handle multiple conversations simultaneously  
- **Event-driven** - React to tool calls, errors, and state changes
- **Observability** - Track what agents are doing and why
- **Error resilience** - Graceful handling of failures

**The Event Handler Pattern**:
This is how you build production agent systems. Instead of waiting for completion, you handle each event as it happens - exactly like modern web applications.

In [27]:
from typing import Any, Set, List
from azure.ai.agents.models import (
    AsyncAgentEventHandler,
    MessageDeltaChunk,
    RunStepDeltaChunk,
    ThreadMessage,
    ThreadRun,
    RunStep,
)

class ProductionEventHandler(AsyncAgentEventHandler):
    """Production-ready event handler with comprehensive tracking."""

    def __init__(self) -> None:
        super().__init__()
        self.tokens: List[str] = []
        self.tool_calls: List[Dict[str, Any]] = []
        self.seen_tools: Set[str] = set()
        self.conversation_complete = False

    async def on_message_delta(self, delta: MessageDeltaChunk) -> None:
        """Capture each token as it's generated - for streaming UI."""
        if delta.text:
            self.tokens.append(delta.text)
            print(delta.text, end="", flush=True)

    async def on_run_step(self, step: RunStep) -> None:
        """Handle completed run steps - track tool execution."""
        # Check for tool calls in the step
        if hasattr(step, 'step_details') and step.step_details:
            step_details = step.step_details
            
            # Handle function tool calls
            if hasattr(step_details, 'tool_calls') and step_details.tool_calls:
                for call in step_details.tool_calls:
                    if hasattr(call, 'function') and call.function:
                        function_name = call.function.name
                        if call.id not in self.seen_tools:
                            self.seen_tools.add(call.id)
                            
                            # Track tool usage for analytics
                            self.tool_calls.append({
                                "call_id": call.id,
                                "function": function_name,
                                "status": str(step.status),
                                "timestamp": time.time()
                            })
                            
                            print(f"\n🛠️  Tool executed: {function_name}", flush=True)
        
        print(f"\n📊 Step status: {step.status}", flush=True)

    async def on_run_step_delta(self, chunk: RunStepDeltaChunk) -> None:
        """Handle tool call progress updates."""
        if chunk.delta:
            delta = chunk.delta
            
            # Extract function name from step details
            step_details = delta.get("step_details", {})
            tool_calls = step_details.get("tool_calls", [])
            
            if tool_calls and len(tool_calls) > 0:
                # Get the latest tool call
                tool_call = tool_calls[0] if isinstance(tool_calls, list) else tool_calls
                if isinstance(tool_call, dict):
                    function_info = tool_call.get("function", {})
                    fn_name = function_info.get("name", "tool")
                else:
                    fn_name = "tool"
            else:
                # Fallback extraction methods
                fn_name = (
                    delta.get("function_call", {}).get("name") or 
                    delta.get("tool_call", {}).get("function", {}).get("name") or
                    "tool"
                )
            
            if chunk.id not in self.seen_tools:
                self.seen_tools.add(chunk.id)
                print(f"\n🔄 Starting: {fn_name}...", flush=True)

    async def on_thread_run(self, run: ThreadRun) -> None:
        """Track overall run status changes."""
        print(f"\n🎯 Run status → {run.status}", flush=True)

    async def on_thread_message(self, message: ThreadMessage) -> None:
        """Track message status updates."""
        if hasattr(message, 'status') and message.status:
            print(f"\n💬 Message → {message.status}", flush=True)

    async def on_error(self, data: str) -> None:
        """Handle errors immediately."""
        print(f"\n🚨 ERROR: {data}", flush=True)

    async def on_done(self) -> None:
        """Conversation completion."""
        self.conversation_complete = True
        print(f"\n✅ Conversation complete", flush=True)
        
        # Summary for production monitoring
        print(f"\n📈 Session Summary:")
        print(f"   • Tokens generated: {len(self.tokens)}")
        print(f"   • Tools called: {len(self.tool_calls)}")
        if self.tool_calls:
            print(f"   • Tool usage: {[call['function'] for call in self.tool_calls]}")

    def get_full_response(self) -> str:
        """Get complete agent response."""
        return "".join(self.tokens)

    def get_tool_summary(self) -> List[Dict[str, Any]]:
        """Get tool execution summary for analytics."""
        return self.tool_calls

print("✅ Production event handler ready for comprehensive agent monitoring")

✅ Production event handler ready for comprehensive agent monitoring



In [28]:
import functools
from typing import Optional, Dict
from azure.ai.agents.aio import AgentsClient
from azure.ai.agents.models import AsyncAgentEventHandler

async def create_async_agent(client: AgentsClient, *, model: str, name: str, instructions: str, toolset: ToolSet) -> str:
    """Create agent with async client."""
    agent = await client.create_agent(model=model, name=name, instructions=instructions, toolset=toolset)
    logger.info("Async agent created → %s", agent.id)
    return agent.id

async def run_agent_with_monitoring(
    client: AgentsClient,
    agent_id: str,
    user_prompt: str,
    event_handler: Optional[AsyncAgentEventHandler] = None,
) -> Dict[str, Any]:
    """Execute agent with comprehensive event monitoring."""
    
    handler = event_handler or ProductionEventHandler()
    
    # Create conversation thread
    thread = await client.threads.create()
    await client.messages.create(thread_id=thread.id, role="user", content=user_prompt)
    
    print(f"🚀 Starting agent execution...")
    print(f"📝 User prompt: {user_prompt[:100]}{'...' if len(user_prompt) > 100 else ''}")
    print(f"🎯 Agent: {agent_id}")
    print("─" * 60)
    
    start_time = time.time()
    
    # Stream execution with event monitoring
    async with await client.runs.stream(
        thread_id=thread.id, 
        agent_id=agent_id, 
        event_handler=handler
    ) as stream:
        await stream.until_done()
    
    execution_time = time.time() - start_time
    
    return {
        "response": handler.get_full_response() if hasattr(handler, 'get_full_response') else "",
        "tool_calls": handler.get_tool_summary() if hasattr(handler, 'get_tool_summary') else [],
        "execution_time": execution_time,
        "thread_id": thread.id
    }

print("✅ Async agent execution framework ready")

✅ Async agent execution framework ready



In [29]:
async def demo_complete_agent_cycle():
    """Demonstrate the complete 5-stage agent architecture."""
    
    # Use the same JSON wrapper pattern as the working sync version
    safe_tools = {json_safe_wrapper(tool) for tool in AGENT_TOOLS}
    
    # Create toolset
    function_tool = FunctionTool(safe_tools)
    toolset = ToolSet()
    toolset.add(function_tool)
    
    # Use consistent environment variable pattern from working version
    endpoint = os.getenv("AZURE_AI_FOUNDRY_URL") or os.getenv("AZURE_AI_FOUNDRY_ENDPOINT")
    if not endpoint:
        print("❌ Missing AZURE_AI_FOUNDRY_URL or AZURE_AI_FOUNDRY_ENDPOINT")
        return
    
    # Use the same model deployment logic as working version
    model = get_model_deployment()
    
    # Create async client (not with statement - match sync pattern)
    credential = DefaultAzureCredential()
    client = AgentsClient(endpoint=endpoint, credential=credential)
    
    try:
        # Create production agent
        agent_id = await create_async_agent(
            client,
            model=model,
            name="production-demo-agent",
            instructions="""You are a helpful business analyst agent. 
            When given tasks, use your available tools to provide comprehensive analysis.
            Always explain your reasoning and show how you used the tools.""",
            toolset=toolset,
        )
        
        # Complex business scenario
        business_prompt = """
        I need a comprehensive analysis for a quarterly business review:
        
        1. Get the current MSFT stock price
        2. Analyze the sentiment of this customer feedback: 'Microsoft Azure has been absolutely amazing for our cloud migration. The AI services exceeded our expectations and the support team was excellent.'
        3. Calculate metrics for our quarterly revenue figures: [125.5, 142.3, 158.7, 167.2]
        4. Summarize your findings in 150 words
        
        Show me your step-by-step analysis.
        """
        
        # Execute with monitoring
        print("🎯 DEMONSTRATING COMPLETE AGENT ARCHITECTURE")
        print("=" * 70)
        
        result = await run_agent_with_monitoring(
            client, 
            agent_id, 
            business_prompt, 
            ProductionEventHandler()
        )
        
        # Analysis summary
        print("\n" + "=" * 70)
        print("📊 EXECUTION ANALYSIS:")
        print(f"⏱️  Execution time: {result['execution_time']:.2f} seconds")
        print(f"🛠️  Tools used: {len(result['tool_calls'])}")
        for tool in result['tool_calls']:
            print(f"   • {tool['function']} (status: {tool['status']})")
        
        print(f"\n📝 AGENT RESPONSE LENGTH: {len(result['response'])} characters")
        
        # Clean up
        await client.delete_agent(agent_id)
        logger.info("Demo agent deleted")
        
        return result
        
    finally:
        # Ensure client is properly closed
        await client.close()

# Execute the complete demo
print("🚀 Starting complete agent architecture demonstration...")
try:
    demo_result = await demo_complete_agent_cycle()
    print("\n✅ Agent architecture demo completed successfully!")
except Exception as e:
    print(f"\n❌ Demo failed: {e}")
    logger.error("Demo execution failed", exc_info=True)

🚀 Starting complete agent architecture demonstration...



INFO:azure.identity._credentials.environment:No environment configuration found.
INFO:azure.identity._credentials.managed_identity:ManagedIdentityCredential will use IMDS
INFO:azure.identity._credentials.managed_identity:ManagedIdentityCredential will use IMDS
INFO:azure.core.pipeline.policies.http_logging_policy:Request URL: 'http://169.254.169.254/metadata/identity/oauth2/token?api-version=REDACTED&resource=REDACTED'
Request method: 'GET'
Request headers:
    'User-Agent': 'azsdk-python-identity/1.19.0 Python/3.11.11 (Windows-10-10.0.26100-SP0)'
No body was attached to the request
INFO:azure.core.pipeline.policies.http_logging_policy:Request URL: 'http://169.254.169.254/metadata/identity/oauth2/token?api-version=REDACTED&resource=REDACTED'
Request method: 'GET'
Request headers:
    'User-Agent': 'azsdk-python-identity/1.19.0 Python/3.11.11 (Windows-10-10.0.26100-SP0)'
No body was attached to the request
INFO:azure.identity._credentials.chained:DefaultAzureCredential acquired a token 

🎯 DEMONSTRATING COMPLETE AGENT ARCHITECTURE



INFO:azure.core.pipeline.policies.http_logging_policy:Request URL: 'https://poc-ai-agents-voice-resource.services.ai.azure.com/api/projects/poc-ai-agents-voice/threads?api-version=REDACTED'
Request method: 'POST'
Request headers:
    'Content-Type': 'application/json'
    'Content-Length': '2'
    'Accept': 'application/json'
    'x-ms-client-request-id': '1e5d0393-8857-11f0-820b-f43bd8cfe843'
    'User-Agent': 'azsdk-python-ai-agents/1.1.0 Python/3.11.11 (Windows-10-10.0.26100-SP0)'
    'Authorization': 'REDACTED'
A body is sent with the request
INFO:azure.core.pipeline.policies.http_logging_policy:Response status: 200
Response headers:
    'Content-Length': '137'
    'Content-Type': 'application/json'
    'Request-Context': 'REDACTED'
    'x-ms-response-type': 'REDACTED'
    'x-ms-middleware-request-id': 'REDACTED'
    'openai-version': 'REDACTED'
    'openai-organization': 'REDACTED'
    'X-Request-ID': 'REDACTED'
    'openai-processing-ms': 'REDACTED'
    'Strict-Transport-Security

🚀 Starting agent execution...
📝 User prompt: 
        I need a comprehensive analysis for a quarterly business review:

        1. Get the curren...
🎯 Agent: asst_5o7LPUEHv2I4za8QsYlTmrwS
────────────────────────────────────────────────────────────

📝 User prompt: 
        I need a comprehensive analysis for a quarterly business review:

        1. Get the curren...
🎯 Agent: asst_5o7LPUEHv2I4za8QsYlTmrwS
────────────────────────────────────────────────────────────


INFO:azure.core.pipeline.policies.http_logging_policy:Request URL: 'https://poc-ai-agents-voice-resource.services.ai.azure.com/api/projects/poc-ai-agents-voice/threads/thread_5Y1cSH58CUleZBtidqtS0Xxl/runs?api-version=REDACTED'
Request method: 'POST'
Request headers:
    'Content-Type': 'application/json'
    'Content-Length': '65'
    'Accept': 'application/json'
    'x-ms-client-request-id': '1eb0ef38-8857-11f0-b0f7-f43bd8cfe843'
    'User-Agent': 'azsdk-python-ai-agents/1.1.0 Python/3.11.11 (Windows-10-10.0.26100-SP0)'
    'Authorization': 'REDACTED'
A body is sent with the request
INFO:azure.core.pipeline.policies.http_logging_policy:Response status: 200
Response headers:
    'Transfer-Encoding': 'chunked'
    'Content-Type': 'text/event-stream; charset=utf-8'
    'Request-Context': 'REDACTED'
    'x-ms-response-type': 'REDACTED'
    'x-ms-middleware-request-id': 'REDACTED'
    'openai-version': 'REDACTED'
    'openai-organization': 'REDACTED'
    'X-Request-ID': 'REDACTED'
    'ope


🎯 Run status → RunStatus.QUEUED

🎯 Run status → RunStatus.QUEUED


🎯 Run status → RunStatus.QUEUED

🎯 Run status → RunStatus.IN_PROGRESS

🎯 Run status → RunStatus.IN_PROGRESS

📊 Step status: RunStepStatus.IN_PROGRESS

📊 Step status: RunStepStatus.IN_PROGRESS

📊 Step status: RunStepStatus.IN_PROGRESS

📊 Step status: RunStepStatus.IN_PROGRESS

🔄 Starting: tool...

🔄 Starting: tool...

🎯 Run status → RunStatus.REQUIRES_ACTION

🎯 Run status → RunStatus.REQUIRES_ACTION

✅ Conversation complete

📈 Session Summary:
   • Tokens generated: 0
   • Tools called: 0

📊 EXECUTION ANALYSIS:

✅ Conversation complete

📈 Session Summary:
   • Tokens generated: 0
   • Tools called: 0

📊 EXECUTION ANALYSIS:
⏱️  Execution time: 2.47 seconds
🛠️  Tools used: 0

📝 AGENT RESPONSE LENGTH: 0 characters
⏱️  Execution time: 2.47 seconds
🛠️  Tools used: 0

📝 AGENT RESPONSE LENGTH: 0 characters


INFO:azure.core.pipeline.policies.http_logging_policy:Request URL: 'https://poc-ai-agents-voice-resource.services.ai.azure.com/api/projects/poc-ai-agents-voice/assistants/asst_5o7LPUEHv2I4za8QsYlTmrwS?api-version=REDACTED'
Request method: 'DELETE'
Request headers:
    'Accept': 'application/json'
    'x-ms-client-request-id': '202c212d-8857-11f0-aaa2-f43bd8cfe843'
    'User-Agent': 'azsdk-python-ai-agents/1.1.0 Python/3.11.11 (Windows-10-10.0.26100-SP0)'
    'Authorization': 'REDACTED'
No body was attached to the request
INFO:azure.core.pipeline.policies.http_logging_policy:Response status: 200
Response headers:
    'Content-Length': '95'
    'Content-Type': 'application/json'
    'Request-Context': 'REDACTED'
    'x-ms-response-type': 'REDACTED'
    'x-ms-middleware-request-id': 'REDACTED'
    'openai-version': 'REDACTED'
    'openai-organization': 'REDACTED'
    'X-Request-ID': 'REDACTED'
    'openai-processing-ms': 'REDACTED'
    'Strict-Transport-Security': 'REDACTED'
    'apim-req


✅ Agent architecture demo completed successfully!

